In [1]:
import pandas as pd
import datetime
import re
import requests
from requests import get
from bs4 import BeautifulSoup
import warnings
from time import sleep
from random import randint
from time import time
timestart_time = time()
warnings.warn('Warning Simulation')
from IPython.core.display import clear_output

C:\Users\k.kiki\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Warning Simulation
  if sys.path[0] == '':


In [2]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
raw_html = requests.get('https://eody.gov.gr/category/deltia-typoy/')
soup = BeautifulSoup(raw_html.content, "html.parser") 
articles_container = soup.find('div',{'class':'articles-container'})
titles = articles_container.find_all('a', href=True)

# if 'νακοίνωση για την εξέλιξη της νόσου COVID-19 στη χώρα μας' in titles[0].text:
#     print(titles[0].text,titles[0].get('href'))
# else:
#     print(titles[0].text,titles[0].get('href')+', but you do not care about this announcement!!!')

frame = []

for t in titles:
    entries = {}
    
    if 'νακοίνωση για την εξέλιξη της νόσου COVID-19 στη χώρα μας' in t.text\
        or 'νημέρωση διαπιστευμένων συντακτών υγείας από' in t.text:

        entries['title'] = t.text
        entries['url'] = t.get('href')

        frame.append(entries)
    
frame = pd.DataFrame(frame)
frame['date'] = frame.title.str.extract(r'[(](\d+\/\d+\/\d+)[)]')
frame['date'] = pd.to_datetime(frame['date'])
frame['date'] = pd.to_datetime(frame["date"], format="%Y-%m-%d").dt.strftime("%Y-%m-%d")

frame

,title,url,date
0,Ανακοίνωση για την εξέλιξη της νόσου COVID-19 στη χώρα μας (20/5/2020),https://eody.gov.gr/0520_briefing_covid19/,2020-05-20
1,"Eνημέρωση διαπιστευμένων συντακτών υγείας από τον Υφυπουργό Πολιτικής Προστασίας και Διαχείρισης Κρίσεων Νίκο Χαρδαλιά και τον εκπρόσωπο του Υπουργείου Υγείας για το νέο κορονοϊό, Καθηγητή Σωτήρη Τσιόδρα (19/5/2020)",https://eody.gov.gr/0519_briefing_covid19/,2020-05-19
2,Ανακοίνωση για την εξέλιξη της νόσου COVID-19 στη χώρα μας (18/5/2020),https://eody.gov.gr/anakoinosi-gia-tin-exelixi-tis-nosoy-covid-19-sti-chora-mas-18-5-2020/,2020-05-18
3,"Ενημέρωση διαπιστευμένων συντακτών υγείας από τον Υφυπουργό Πολιτικής Προστασίας και Διαχείρισης Κρίσεων Νίκο Χαρδαλιά, τον Παθολόγο-Λοιμωξιολόγο, Υποπτέραρχο Δημήτρη Χατζηγεωργίου και τον Υφυπουργό Ανάπτυξης και Επενδύσεων Νίκο Παπαθανάση (17/5/2020)",https://eody.gov.gr/0517_briefing_covid19/,2020-05-17
4,Ανακοίνωση για την εξέλιξη της νόσου COVID-19 στη χώρα μας (16/5/2020),https://eody.gov.gr/anakoinosi-gia-tin-exelixi-tis-nosoy-covid-19-sti-chora-mas-16-5-2020/,2020-05-16
5,"Ενημέρωση διαπιστευμένων συντακτών υγείας από τον Υφυπουργό Πολιτικής Προστασίας και Διαχείρισης Κρίσεων Νίκο Χαρδαλιά και τον εκπρόσωπο του Υπουργείου Υγείας για το νέο κορονοϊό, Καθηγητή Σωτήρη Τσιόδρα",https://eody.gov.gr/enimerosi-diapisteymenon-syntakton-ygeias-apo-ton-yfypoyrgo-politikis-prostasias-kai-diacheirisis-kriseon-niko-chardalia-kai-ton-ekprosopo-toy-ypoyrgeioy-ygeias-gia-to-neo-koronoio-kathigiti-sotir-7/,NaT


In [4]:
def get_press_release(row):
    page = requests.get(row['url'])
    soup_doc = BeautifulSoup(page.content, "html.parser") 
    
    par = ''
    for p in soup_doc.find_all('p'):
        par = par+p.text.strip()
        
    return pd.Series({
    'content': par 
    })   

In [5]:
df = frame.apply(get_press_release, axis=1).join(frame)
# df.head()

In [6]:
df['new_cases'] = df.content.str.extract(r'.*[aα]νακοινώνο\w+\s+(\d+)\s+νέα\s+κρούσματα',re.IGNORECASE).astype(float)
df['total_cases'] = df.content.str.extract(r'συνολικός\s+αριθμός.*κρουσμάτων\s+είναι\s+(\d+[.]?\d+)',re.IGNORECASE)
df['total_cases'] = df.total_cases.str.replace('.','').astype(float)

In [7]:
# df['new_deaths'] = df.content.str.extract(r'(?:μεθ.*\s+)(\w+|\d+)(?:\s+ακόμα\s+καταγεγραμμένο.*θ[άα]ν[αά]το.*και)',re.IGNORECASE)
#σήμερα καταγράφουμε μηδέν θανάτους και συνολικά στη χώρα έχουμε καταγράψει
df['new_deaths'] = df.content.str.extract(r'(?:μεθ.*\w+με\s+|μεθ.*\w+με\s+ακόμα\s+)(\w+)(?:\s+καταγεγραμμένο.*θ[άα]ν[αά]το.*\s+και.*στη\s+χώρα.*|\s+ακόμα\s+καταγεγραμμένο.*θ[άα]ν[αά]το.*\s+και.*στη\s+χώρα.*|\s+νέο.*θ[άα]ν[αά]το.*και.*στη\s+χώρα.*|\s+θ[άα]ν[αά]το.*και.*στη\s+χώρα.*)',re.IGNORECASE)

In [8]:
df['new_deaths'] = df.new_deaths.replace({'μηδέν':'0',
                                          'κανέναν':'0',
                                          'κανένα':'0',
                                          "έναν": "1", 
                                          "δύο": "2", 
                                          'τρεις':'3', 
                                          'τέσσερις':'4',
                                         'πέντε':'5',
                                         'έξι':'6',
                                         'εφτά':'7',
                                         'επτά':'7',
                                         'οχτώ':'8',
                                         'οκτώ':'8',
                                         'εννέα':'9',
                                         'ενιά':'9',
                                         'δέκα':'10',
                                         'έντεκα':'11',
                                         'δώδεκα':'12'}).astype(float)

In [9]:
df['total_deaths'] = df.content.str.extract(\
                    r'(?:.*θ[άα]ν[αά]το.*και.*?\s+)(\d+).*?\s+θ[άα]ν[αά]τους.*?\s+στη\s+χώρα',\
                                           re.IGNORECASE).astype(float)

In [10]:
df['intubated'] = df.content.str.extract(r'(\d+)\s+συμπολίτες\s+μας\s+νοσηλεύονται\s+διασωληνωμένοι').astype(float)

In [11]:
df['total_tests'] = df.content.str.extract(r'.*\s+ελεγχθεί\s+(\d+[.]?\d+)\s+κλινικά\s+δείγματα',re.IGNORECASE)
df['total_tests'] = df.total_tests.str.replace('.','').astype(float)

In [12]:
# df.head()

In [13]:
# df.new_cases.value_counts(dropna=False)

In [14]:
# df.total_cases.value_counts(dropna=False)

In [15]:
# df.new_deaths.value_counts(dropna=False)

In [16]:
# df.total_deaths.value_counts(dropna=False)

In [17]:
# df.intubated.value_counts(dropna=False)

In [18]:
# df.total_tests.value_counts(dropna=False)

In [19]:
# df[df.total_tests.isnull()]

In [20]:
greece = pd.read_csv('https://raw.githubusercontent.com/iMEdD-Lab/open-data/master/COVID-19/greece.csv')
# greece.head()

In [21]:
if df['date'][0] == pd.datetime.today().strftime("%Y-%m-%d"):


    if greece.cases.sum() < df.total_cases[0] and greece.cases.sum()+df.new_cases[0] == df.total_cases[0]:
        greece[greece['county_en'] == 'Unknown']['cases'] = greece[greece['county_en'] == 'Unknown']['cases']+(df.total_cases[0]-greece.cases.sum())
        greece.to_csv('greece_auto_test.csv')
        print('greece.cases was updated and greece_auto_test.csv has been saved')

    elif greece.cases.sum() < df.total_cases[0] and greece.cases.sum()+df.new_cases[0] != df.total_cases[0]:
        print('Be careful: Something goes wrong with cases announced!!!')

    elif greece.cases.sum() >= df.total_cases[0]:
        print('Cool: greece.cases is updated!')
        
    else:
        print('Sorry, I do not know what is happening with greece.cases!')



    if greece.dead.sum() < df.total_deaths[0] and greece.dead.sum()+df.new_deaths[0] == df.total_deaths[0]:
        greece[greece['county_en'] == 'Unknown']['dead'] = greece[greece['county_en'] == 'Unknown']['cases']+df.new_deaths[0]
        greece.to_csv('greece_auto_test.csv')
        print('greece.dead was updated and greece_auto_test.csv has been saved')

    elif greece.dead.sum() < df.total_deaths[0] and greece.dead.sum()+df.new_deaths[0] != df.total_deaths[0]:
        print('Be careful: Something goes wrong with deaths announced!!!')
        
    elif greece.dead.sum() >= df.total_deaths[0]:
        print('Cool: greece.dead is updated!')

    else:
        print('Sorry, I do not know what is happening with greece.dead!')


    if greece.critical.sum() != df.intubated[0]:
        greece[greece['county_en'] == 'Unknown']['critical'] = greece[greece['county_en'] == 'Unknown']['critical']+(df.intubated[0]-greece.critical.sum())
        greece.to_csv('greece_auto_test.csv')
        print('greece.critical was updated and greece_auto_test.csv has been saved')

    elif greece.critical.sum() == df.intubated[0]:
        print('Cool: greece.critical is updated!')
        
    else:
        print('Sorry, I do not know what is happening with greece.critical!')

        
else:
    print('Latest announcement published refers to '+str(df['date'][0])+'. So, you should be updated.')


Latest announcement published refers to 2020-05-20. So, you should be updated.


In [22]:
greeceTimeline = pd.read_csv('https://raw.githubusercontent.com/iMEdD-Lab/open-data/master/COVID-19/greeceTimeline.csv')
# greeceTimeline = greeceTimeline.drop(greeceTimeline.columns[-1],axis=1)
grt = greeceTimeline
grt = grt.drop(["Province/State", "Country/Region"], axis=1)
grt = grt.set_index("Status")
grt = grt.T
grt = grt.reset_index()
grt = grt.rename(columns={"index": "date"})
grt['date'] = pd.to_datetime(grt['date']).dt.strftime("%Y-%m-%d")

grt.tail()

Status,date,cases,cruise_ship_ElVen,Ritsona,deaths,recovered,hospitalized,intensive_care,intubated,total_tests,total cases
80,2020-05-16,9.0,0.0,0.0,1.0,NaN,NaN,NaN,23.0,126283.0,2819.0
81,2020-05-17,15.0,0.0,0.0,1.0,NaN,NaN,NaN,22.0,128525.0,2834.0
82,2020-05-18,2.0,0.0,0.0,2.0,NaN,NaN,NaN,24.0,131684.0,2836.0
83,2020-05-19,4.0,0.0,0.0,1.0,NaN,NaN,NaN,22.0,136001.0,2840.0
84,2020-05-20,10.0,0.0,0.0,1.0,NaN,NaN,NaN,22.0,139445.0,2850.0


In [23]:
timeline = {}
# if grt['date'].iloc[-1] < df['date'][0]:
if grt['date'].iloc[-1] < pd.datetime.today().strftime("%Y-%m-%d") and df['date'][0] == pd.datetime.today().strftime("%Y-%m-%d"):

    timeline['date'] = df['date'][0]
    timeline['cases'] = df['new_cases'][0]
    timeline['total cases'] = df['total_cases'][0]
    timeline['deaths'] = df['new_deaths'][0]
    timeline['intubated'] = df['intubated'][0]
    timeline['total_tests'] = df['total_tests'][0]
     
    grt = grt.append(timeline,ignore_index=True)
    
    grt['date'] = grt.date.astype(str)
    grt['year'] = grt.date.str.extract(r'(\d\d)\d\d-\d\d-\d\d')
    grt['month'] = grt.date.str.extract(r'\d\d\d\d-(\d\d)-\d\d')
    grt['month'] = grt.month.replace({'01':'1',
                                   '02':'2',
                                   '03':'3',
                                   '04':'4',
                                   '05':'5',
                                   '06':'6',
                                   '07':'7',
                                   '08':'8',
                                   '09':'9'})
    grt['day'] = grt.date.str.extract(r'\d\d\d\d-\d\d-(\d\d)')
    grt['day'] = grt.day.replace({'01':'1',
                               '02':'2',
                               '03':'3',
                               '04':'4',
                               '05':'5',
                               '06':'6',
                               '07':'7',
                               '08':'8',
                               '09':'9'})
    grt['date2'] = grt.month+'/'+grt.day+'/'+grt.year
    grt = grt.drop(['date','year','month','day'],axis=1)
    
    grt = grt.set_index('date2').T.reset_index()
    grt = grt[['Status',grt.columns[-1]]].copy()
    grt_new = pd.merge(greeceTimeline,grt,on='Status',how='outer')

    grt_new.to_csv('greeceTimeline_auto_test.csv')
    print('Daily update: greeceTimeline appended and greece_auto_test.csv has been saved')

elif grt['date'].iloc[-1] < pd.datetime.today().strftime("%Y-%m-%d") and df['date'][0] != pd.datetime.today().strftime("%Y-%m-%d"):
    
    print('greece.Timeline is not updated, because no announcement has been out for today. Please wait or check out what is happening!')
    
elif grt['date'].iloc[-1] == pd.datetime.today().strftime("%Y-%m-%d"):
    
    print('Cool: greece.Timeline is updated!')

else: 
    print('Sorry, I do not know what is happening with greece.Timeline!')


#grt_new

greece.Timeline is not updated, because no announcement has been out for today. Please wait or check out what is happening!
